In [5]:
#Import here
from sentence_transformers import SentenceTransformer, util
import torch
from getDocument import *

In [ ]:
from transformers import AutoModel, AutoTokenizer, BertModel

model = BertModel.from_pretrained("allenai/scibert_scivocab_uncased", is_decoder=True) #add_cross_attention
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', padding=True, truncation=True) 

The following is an example of running Torch.nn.CosineSimilarity() on single sentences. It works somewhat as text1 and text2 have a higher similarity than 1/3 and 2/3.  Torch.nn.CosineSimilarity(dim=2) returns a vector of size [1, 14] which I then sum to get the similarity *THIS MAY NOT BE THE CORRECT WAY TO DO IT* but right now I don't have any better guidance.

In [ ]:
# Test of sentences to gather understanding of what cosine similarity is
# https://pytorch.org/docs/stable/generated/torch.nn.CosineSimilarity.html

text1 = "No one was allowed to interfere with the work."
text2 = "The job was done with no interaction."
text3 = "Walking in the park is fun when it's warm."

MAX_LENGTH = 18

# Need to do the padding buisness because nn.CosineSimilarity requires tensors of equal length
encoded_input1 = tokenizer(text1, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_input2 = tokenizer(text2, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_input3 = tokenizer(text3, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')

output1 = model(**encoded_input1, return_dict=False)
output2 = model(**encoded_input2, return_dict=False)
output3 = model(**encoded_input3, return_dict=False)

print(f"encoded_input1: {encoded_input1['input_ids']}")
print(f"encoded_input2: {encoded_input2['input_ids']}")
print(f"encoded_input3: {encoded_input3['input_ids']}")
print(f"decode of encoded_input1: {tokenizer.decode(encoded_input1['input_ids'][0])}")
print(f"decode of encoded_input2: {tokenizer.decode(encoded_input2['input_ids'][0])}")
print(f"decode of encoded_inpu32: {tokenizer.decode(encoded_input3['input_ids'][0])}")

# print(output1[0].shape)
# print(output2[0].shape)
# print(output3[0].shape)

# cosine_scores = util.cos_sim(encoded_input1["input_ids"], encoded_input2["input_ids"])
cosine_scores_DIM2 = torch.nn.CosineSimilarity(dim=2)
# cosine_scores2 = torch.nn.CosineSimilarity(dim=1)
# cosine_scores3 = torch.nn.CosineSimilarity(dim=2)

score1 = cosine_scores_DIM2(output1[0], output2[0])
score2 = cosine_scores_DIM2(output1[0], output3[0])
score3 = cosine_scores_DIM2(output2[0], output3[0])
# score4 = cosine_scores4(output1[0], output2[0])

# print(f"score1.shape: {score1.shape}")
# print(f"score2.shape: {score2.shape}")
# print(f"score3.shape: {score3.shape}")

# print(f"sum of score3: {score3[0].sum().item()} and score3[0].size()[0]: {score3[0].size()[0]}")
print(f"Sentence 1 & 2 cosine similarity score: {score1.mean().item()}")
print(f"Sentence 1 & 3 cosine similarity score: {score2.mean().item()}")
print(f"Sentence 2 & 3 cosine similarity score: {score3.mean().item()}")

With max_length=14 the scores are
1/2 -> 0.582
1/3 -> 0.475
2/3 -> 0.490

With max_length = 18 the scores are
1/2 -> 0.656
1/3 -> 0.573
2/3 -> 0.588

seeming to suggest that unnecessary padding will hurt the accuracy (as expected)
_____________________________________________________________________________________________________________________

In [ ]:
paragraph1 = "Pre-trained text encoders have rapidly advanced the state of the art on many NLPtasks. We focus on one such model, BERT, and aim to quantify where linguistic information is captured within the network. We find that the model represents the steps of the traditional NLP pipeline in an interpretable and localizable way, and that the regions responsible for each step appear in the expected sequence: POS tagging, parsing, NER, semantic roles, then coreference. Qualitative analysis reveals that the model can and often does adjust this pipeline dynamically, revising lower-level decisions on the basis of disambiguating information from higher-level representations."
paragraph2 = "We present a set of experiments to demonstrate that deep recurrent neural networks (RNNs) learn internal representations that capture soft hierarchical notions of syntax from highly varied supervision. We consider four syntax tasks at different depths of the parse tree; for each word, we predict its part of speech as well as the first (parent), second (grandparent) and third level (great-grandparent) constituent labels that appear above it. These predictions are made from representations produced at different depths in networks that are pretrained with one of four objectives: dependency parsing, semantic role labeling, machine translation, or language modeling. In every case, we find a correspondence between network depth and syntactic depth, suggesting that a soft syntactic hierarchy emerges. This effect is robust across all conditions, indicating that the models encode significant amounts of syntax even in the absence of an explicit syntactic training supervision."
paragraph3 = "Language  technology  has  become  pervasive  in  everyday  life. Neural  networks  are  a key component  in  this technology  thanks  to  their ability  to model  large  amounts  of data.  Contrary  to traditional  systems,  models  based  on  deep  neural  networks  (a.k.a. deep  learning) can  be  trained  in  an  end-to-end  fashion  on  input-output  pairs,  such  as  a  sentence  in  one language  and  its translation  in  another  language,  or  a  speech  utterance  and  its  transcription.  The end-to-end  training  paradigm simplifies  the  engineering process  while  giving  the model  flexibility  to  optimize  for  the  desired  task. This,  however,  often  comes  at  the  expense  of model  interpretability:  understanding  the role  of different parts  of the  deep  neural network  is  difficult,  and  such  models  are  sometimes  perceived  as  'black-box',  hindering research  efforts  and  limiting their  utility to  society."

MAX_LENGTH = 175

# Need to do the padding buisness because nn.CosineSimilarity requires tensors of equal length
encoded_long_input1 = tokenizer(paragraph1, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_long_input2 = tokenizer(paragraph2, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')
encoded_long_input3 = tokenizer(paragraph3, return_tensors='pt', max_length=MAX_LENGTH, padding='max_length')

output_long_1 = model(**encoded_long_input1, return_dict=False)
output_long_2 = model(**encoded_long_input2, return_dict=False)
output_long_3 = model(**encoded_long_input3, return_dict=False)

# print(f"encoded_input1: {encoded_long_input1['input_ids']}")
# print(f"encoded_input2: {encoded_long_input2['input_ids']}")
# print(f"encoded_input3: {encoded_long_input3['input_ids']}")
print(f"decode of encoded_input1: {tokenizer.decode(encoded_long_input1['input_ids'][0])}")
print(f"decode of encoded_input2: {tokenizer.decode(encoded_long_input2['input_ids'][0])}")
print(f"decode of encoded_inpu32: {tokenizer.decode(encoded_long_input3['input_ids'][0])}")

print(output_long_1[0].shape)
print(output_long_2[0].shape)
print(output_long_3[0].shape)

cosine_scores_DIM2 = torch.nn.CosineSimilarity(dim=0)

score_long_1 = cosine_scores_DIM2(output_long_1[0], output_long_2[0])
score_long_2 = cosine_scores_DIM2(output_long_1[0], output_long_3[0])
score_long_3 = cosine_scores_DIM2(output_long_2[0], output_long_3[0])

print(f"Sentence 1 & 2 cosine similarity score: {score_long_1.mean().item()}")
print(f"Sentence 1 & 3 cosine similarity score: {score_long_2.mean().item()}")
print(f"Sentence 2 & 3 cosine similarity score: {score_long_3.mean().item()}")


Text came from the BERT&ClassicNLP folder with 1: main paper, 2: source1, 3: source2
using their abstracts, source 3 is supposed to be a semi-black sheep source as it's a thesis abstract and not a normal academic paper one

With max_length=175 & dim=2:
Sentence 1 & 2 cosine similarity score: 0.296
Sentence 1 & 3 cosine similarity score: 0.364
Sentence 2 & 3 cosine similarity score: 0.319

With max_length=175 & dim=1:
Sentence 1 & 2 cosine similarity score: 0.224
Sentence 1 & 3 cosine similarity score: 0.248
Sentence 2 & 3 cosine similarity score: 0.237

With max_length=175 & dim=0:
Sentence 1 & 2 cosine similarity score: 0.191
Sentence 1 & 3 cosine similarity score: 0.219
Sentence 2 & 3 cosine similarity score: 0.201

______________________________________________________________________________________________________________________

Get the tuple of SPECTER embeddings of a main paper and all of its references

Both indices of the tuple are lists, 0 is embeddings and 1 is references

The 0 index of each list is of the main paper

The each embedding is just a list of numbers

In [ ]:
tupleOfResults = get_paper_and_references_embedding_and_titles_from_query("BERT Rediscovers the Classical NLP Pipeline")

embeddings = tupleOfResults[0]
titles = tupleOfResults[1]

print(titles)

In [ ]:
# Test the Cosine Similarity by first creating tensors and comparing the results
newCosineSimilarity = torch.nn.CosineSimilarity(dim=0)
mainPaper = torch.tensor(embeddings[0])
print(f"Main paper title: {titles[0]}")
for i in range(1, len(embeddings)):
    referencePaper = torch.tensor(embeddings[i])
    print(f"Main Paper with embeddings[{i}]: {newCosineSimilarity(mainPaper, referencePaper):.3f} and the reference paper: {titles[i]}")


With dim=0: (Which is the only allowed cosine similarity )
Main Paper with embeddings[1]: 0.7507858276367188
Main Paper with embeddings[2]: 0.7007443308830261
Main Paper with embeddings[3]: 0.734921395778656
Main Paper with embeddings[4]: 0.7470535039901733
Main Paper with embeddings[5]: 0.7582955956459045
Main Paper with embeddings[6]: 0.6695845127105713
Main Paper with embeddings[7]: 0.7345977425575256
Main Paper with embeddings[8]: 0.6843916773796082
Main Paper with embeddings[9]: 0.7346400618553162
Main Paper with embeddings[10]: 0.7553737759590149
Main Paper with embeddings[11]: 0.6457442045211792
Main Paper with embeddings[12]: 0.7245481610298157
Main Paper with embeddings[13]: 0.6589515209197998
Main Paper with embeddings[14]: 0.6954355835914612
Main Paper with embeddings[15]: 0.6530366539955139
Main Paper with embeddings[16]: 0.5189672708511353
Main Paper with embeddings[17]: 0.6884840726852417
Main Paper with embeddings[18]: 0.616716206073761
Main Paper with embeddings[19]: 0.644980251789093
Main Paper with embeddings[20]: 0.6220344305038452

_______________________________________________________________________________________________________________________________________________
Create a reference paper corpus and then do cosine similarity

In [9]:
defaultPaper = get_paper_and_references_embedding_and_titles_from_query("BERT Rediscovers the Classical NLP Pipeline")

Request failed.


UnboundLocalError: local variable 'content' referenced before assignment

In [8]:
print(defaultPaper)

# get_paper_text_from_paperId(defaultPaper[0][0])

([[-5.712510108947754, -2.7038633823394775, 3.62611985206604, 2.646200180053711, 3.573634624481201, -0.987500011920929, 3.2663674354553223, -0.11620324850082397, 0.7858099937438965, 1.566123604774475, 0.4799612760543823, 1.5585567951202393, 0.3054448068141937, 1.623457431793213, 0.3884057104587555, 0.4954937696456909, -0.37682950496673584, -1.3699588775634766, 4.933783531188965, 3.3637948036193848, -1.0607266426086426, 2.3951187133789062, -2.3635923862457275, -2.197331428527832, -1.702465295791626, -0.7193527817726135, 5.186407089233398, 3.7848920822143555, -5.306734561920166, 0.7330350875854492, -0.842670202255249, -3.1763112545013428, 3.0137271881103516, -3.4549922943115234, 0.651421308517456, -4.824199199676514, -3.281315326690674, 5.7014312744140625, -3.5971620082855225, -0.47249799966812134, -0.6284208297729492, -1.8905751705169678, 2.012805461883545, 2.460982322692871, 0.7342075705528259, 1.851709008216858, 2.3994414806365967, 3.5924794673919678, -2.5355823040008545, 2.0693316459

In [ ]:
# Create a dictionary for the purpose of not repreating titles
title_embedding_dict = {}

for embedding, title in zip(defaultPaper[0], defaultPaper[1]):
    subPaper = get_paper_and_references_embedding_and_titles_from_query(title)
    # First depth, all of the main paper's references
    if title not in title_embedding_dict:
        title_embedding_dict[title] = embedding

    # Second depth, all of the referenced paper's references
    for sub_embedding, sub_title in zip(subPaper[0], subPaper[1]):
        if sub_title not in title_embedding_dict:
            title_embedding_dict[sub_title] = sub_embedding
    

In [ ]:
# A function to return just the first value of the list to be used in list.sort()
def getKey(listOfCosAndTitle):
    return listOfCosAndTitle[0]

In [ ]:
print(len(title_embedding_dict))
# Redefine cos similarity here for ease of use
# also redefine 'main' paper
newCosineSimilarity = torch.nn.CosineSimilarity(dim=0)
mainPaperEmbedding = torch.tensor(defaultPaper[0][0])

similarityScoresWithTitle = []
# Get the cos similarity between the main paper and all of the subpapers
for key in title_embedding_dict.keys():
    compared_paper = torch.tensor(title_embedding_dict[key])
    similarityScoresWithTitle.append([newCosineSimilarity(mainPaperEmbedding, compared_paper), key])


In [ ]:
GREEN = '\033[32m'
RED = '\033[31m'
RESET = '\033[0m'

similarityScoresWithTitle.sort(reverse=True, key=getKey)
for i in similarityScoresWithTitle:
    print(f"Cos similarity score: {i[0]:.3f} and it's {RED + 'not cited in ' if i[1] not in defaultPaper[1] else GREEN + 'cited in     '}{RESET}the main paper with title: {i[1]:20s}")
print(similarityScoresWithTitle)